In [26]:
# imports
##
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

from scipy.sparse import coo_matrix, hstack

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

import tqdm
from tqdm import trange

In [3]:
# Load the data
##
tweets = pd.read_csv("../csv files/ALL_DATA_V2.csv")
tweets.head()

,tweet_id,text,created_at,likes,text length,polarity,sentiment,target,UserName,ScreenName,Location,TweetAt,Sentiment,ID
0,1.419307e+18,face bound border two The covid said hidden,2021-07-25 17:43:34 EAT,0.0,43.0,-0.1666,negative,-1,NaN,NaN,NaN,NaN,NaN,NaN
1,1.419307e+18,I seeing looking like second global going,2021-07-25 17:43:34 EAT,0.0,41.0,0.0000,neutral,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.419307e+18,sentence people intensive care,2021-07-25 17:43:33 EAT,0.0,30.0,0.0000,neutral,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.419307e+18,contagious delta variant surging across nation...,2021-07-25 17:43:31 EAT,0.0,98.0,0.0000,neutral,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1.419307e+18,sentence people intensive care,2021-07-25 17:43:28 EAT,0.0,30.0,0.0000,neutral,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Dropping the missing data points row-wise
##
tweets.dropna(axis=0, subset=['text'], inplace=True)
tweets.reset_index(drop=True, inplace=True)

## Model Building

In [6]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [7]:
# BOW (bag of words)
##
cv = CountVectorizer(analyzer='word', stop_words=stop)

cv.fit(tweets['text'])

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [8]:
# Preview vocabulary and the number of vocab words
##
print(len(cv.vocabulary_))

20417


In [9]:
list(cv.vocabulary_)[:10]

['face',
 'bound',
 'border',
 'two',
 'covid',
 'said',
 'hidden',
 'seeing',
 'looking',
 'like']

In [10]:
# Example transforming a single text
##
print(cv.transform([tweets['text'][0]]))

  (0, 2061)	1
  (0, 2101)	1
  (0, 3986)	1
  (0, 6362)	1
  (0, 8204)	1
  (0, 15444)	1
  (0, 18732)	1


In [12]:
# Example getting the feature name by index
##
cv.get_feature_names()[7591]

'gorgeous'

In [13]:
# Transforming the whole BOW to a sparse matrix
##
bow_text = cv.transform(tweets['text'])

In [14]:
# Non-zero occurrences
##
bow_text.nnz

1413194

In [15]:
# tfidf weighting
##
tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(bow_text)

TfidfTransformer()

In [16]:
# Example transforming a single bow
##
print(tfidf_transformer.transform(cv.transform([tweets['text'][0]])))

  (0, 18732)	0.3540890140004211
  (0, 15444)	0.33005940555536084
  (0, 8204)	0.43847752566461773
  (0, 6362)	0.3557708794716271
  (0, 3986)	0.27601539418774257
  (0, 2101)	0.4403711131831057
  (0, 2061)	0.4203987210029645


In [17]:
# Example getting idf weight of a word
##
tfidf_transformer.idf_[cv.vocabulary_['good']]

4.902696566057591

In [18]:
# Transforming the whole sparse matrix
##
tfidf_text = tfidf_transformer.transform(bow_text)

Adding 'text length' and 'word count' as features to the model  
We'll stack the features to the sparse matrix horizontally

In [31]:
"""# Transforming the two columns into sparse matrices
##
txt_length = coo_matrix(tweets['text length']).reshape(1182,1)
wrd_cnt = coo_matrix(tweets['word count']).reshape(1182,1)

features = hstack([tfidf_text, txt_length, wrd_cnt])

# Preview difference in shapes
print("Shape of text column sparse matrix: ", tfidf_text.shape)
print("Shape of concatenated features sparse matrix: ", features.shape)"""

'# Transforming the two columns into sparse matrices\n##\ntxt_length = coo_matrix(tweets[\'text length\']).reshape(1182,1)\nwrd_cnt = coo_matrix(tweets[\'word count\']).reshape(1182,1)\n\nfeatures = hstack([tfidf_text, txt_length, wrd_cnt])\n\n# Preview difference in shapes\nprint("Shape of text column sparse matrix: ", tfidf_text.shape)\nprint("Shape of concatenated features sparse matrix: ", features.shape)'

In [19]:
features = tfidf_text

## Comparing different classification models:  
- Logistic Regression
- Linear SVC (svm)
- SGD Classifier
- Random Forest Classifier
- Xgboost Classifier
- LGBM Classifier

In [20]:
# Splitting data into train and test splits
##
X_train, X_test, y_train, y_test = train_test_split(features, tweets['target'],stratify=tweets.target, test_size=0.3)

In [21]:
# Initialize models
##
lr = LogisticRegression(C=2.0, class_weight=None, dual=False, max_iter=100)

svc = LinearSVC(C=2.0, class_weight=None, dual=False, max_iter=100)

sgd = SGDClassifier()

rfc = RandomForestClassifier(bootstrap=False, class_weight=None, 
                             criterion='entropy', min_samples_split=6, n_estimators=160, warm_start=False)

xgb = XGBClassifier(objective='multi:softmax', num_class=3)

lgbm = LGBMClassifier(objective='multiclass')

In [23]:
# Train models
models = [lr, svc, enumeratec, xgb, lgbm]

for model in models:
    print(f"\nFitting [{model.__class__.__name__}]")
    model.fit(X_train, y_train)


Fitting [LogisticRegression]

Fitting [LinearSVC]

Fitting [SGDClassifier]

Fitting [RandomForestClassifier]

Fitting [XGBClassifier]
[23:51:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Fitting [LGBMClassifier]


In [24]:
# Make predictions
predictions = []
for model in models:
    print(f"\nPredicting for [{model.__class__.__name__}]")
    predictions.append(model.predict(X_test))


Predicting for [LogisticRegression]

Predicting for [LinearSVC]

Predicting for [SGDClassifier]

Predicting for [RandomForestClassifier]

Predicting for [XGBClassifier]

Predicting for [LGBMClassifier]


In [25]:
# Evaluate models
for i, model in enumerate(models):
    print(f"{model.__class__.__name__} \n----------------------------------")
    print(confusion_matrix(y_test, predictions[i]))
    print(classification_report(y_test, predictions[i]))

LogisticRegression 
----------------------------------
[[ 8660  1494  1820]
 [  618 18343  1190]
 [ 1345  2016 16958]]
              precision    recall  f1-score   support

          -1       0.82      0.72      0.77     11974
           0       0.84      0.91      0.87     20151
           1       0.85      0.83      0.84     20319

    accuracy                           0.84     52444
   macro avg       0.83      0.82      0.83     52444
weighted avg       0.84      0.84      0.84     52444

LinearSVC 
----------------------------------
[[ 8868  1324  1782]
 [  698 18071  1382]
 [ 1442  1941 16936]]
              precision    recall  f1-score   support

          -1       0.81      0.74      0.77     11974
           0       0.85      0.90      0.87     20151
           1       0.84      0.83      0.84     20319

    accuracy                           0.84     52444
   macro avg       0.83      0.82      0.83     52444
weighted avg       0.84      0.84      0.84     52444

SGDClassi

In [38]:
cross_val_score(svc, features, tweets['target'], cv=5)

array([0.97100773, 0.96972406, 0.97372717, 0.96976292, 0.96902449])

In [ ]:
# Making sure that the models are not overfitting using cross val score metric with 5 folds
##
print("Logistic Regression \n------------------------- \nscore= ", cross_val_score(lr, features, tweets['target'], cv=5).mean())
print("\nLinear SVC \n------------------------- \nscore=", cross_val_score(svc, features, tweets['target'], cv=5).mean())
print("\nRandom Forest Classifier \n------------------------- \nscore=", cross_val_score(rfc, features, tweets['target'], cv=5).mean())
print("\nXgboost Classifier \n------------------------- \nscore=", cross_val_score(xgb, features, tweets['target'], cv=5).mean())

Logistic Regression 
------------------------- 
score=  0.9492269631769622

Linear SVC 
------------------------- 
score= 0.9706492731618044


In [38]:
# Random Forest performed best
# That's a good score having in mind that the model is attempting to predict between three category classes

## **Stacking**

In [ ]:
stack = StackingClassifier(estimators=[('rfc', rfc), ('svc', svc)], final_estimator=lr)
stack.fit(X_train, y_train)
pred = stack.predict(X_test)
print(f"{confusion_matrix(y_test, pred)}\n{classification_report(y_test, pred)}")

In [29]:
# saving model
from sklearn.pipeline import Pipeline
import pickle

# retrain
pipe = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC(C=2.0, class_weight=None, dual=False, max_iter=100))])
pipe.fit(tweets.text, tweets.target)
pickle.dump(pipe, open('saved_model.pkl', 'wb'))

In [30]:
# load model
model = pickle.load(open('saved_model.pkl', 'rb'))

In [31]:
model.predict(["I don't know"])

array([0])

In [32]:
np.int(pipe.predict(["I love people."]))

1